In [17]:
import os
import pyodbc
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Agent using user defined functions

The code in this Jupyter Notebook demonstrates the creation and usage of an AI agent using Azure's AI Project Client. The agent is designed to assist with weather-related queries, specifically tracking snowfall in different locations. Below is a breakdown of the key components:

1. **Environment Setup**:
    - The `dotenv` library is used to load environment variables, such as the connection string for the Azure Project Client.
    - The `DefaultAzureCredential` is used for authentication with Azure services.

2. **User-Defined Function**:
    - A custom function `recent_snowfall` is defined to fetch recent snowfall data for a given location. This function returns a JSON string containing the location and snowfall details.

3. **Toolset Initialization**:
    - The `FunctionTool` class is used to wrap the user-defined function, making it accessible to the AI agent.
    - A `ToolSet` object is created, and the function tool is added to it.

4. **Enabling Auto Function Calls**:
    - The `enable_auto_function_calls` method is invoked on the `project_client.agents` object, passing the toolset as an argument. This step is crucial as it allows the agent to automatically call the user-defined functions when needed during its execution. Without this step, the agent would not be able to utilize the provided functions, and the code would fail to execute as intended.

5. **Agent Creation**:
    - The AI agent is created with specific instructions, a model (`gpt-4o-mini`), and the toolset containing the user-defined function. The agent is configured to use the function to answer snowfall-related queries.

6. **Thread and Message Handling**:
    - A thread is created for communication with the agent.
    - A user message is sent to the thread, asking for snowfall information about Denver.
    - The agent processes the message using the provided function and returns the response.

7. **Cleanup**:
    - The agent is deleted after use to free up resources.
    - All messages in the thread are fetched and logged for review.

By enabling auto function calls, the agent can seamlessly integrate the user-defined function into its workflow, ensuring that it can respond to queries effectively. This feature simplifies the process of extending the agent's capabilities with custom logic.

In [18]:
load_dotenv()

True

In [19]:
project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING"),
    )

In [20]:
import json
from typing import Callable, Set, Any

def recent_snowfall(location: str) -> str:
    """
    Fetches recent snowfall totals for a given location.
    :param location: The city name.
    :return: Snowfall details as a JSON string.
    """
    mock_snow_data = {"Seattle": "0 inches", "Denver": "2 inches"}
    snow = mock_snow_data.get(location, "Data not available.")
    return json.dumps({"location": location, "snowfall": snow})

user_functions: Set[Callable[..., Any]] = {
    recent_snowfall,
}

In [21]:
from azure.ai.projects.models._patch import FunctionTool
from azure.ai.projects.models import ToolSet

# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

project_client.agents.enable_auto_function_calls(toolset=toolset)

# Create your agent with the toolset
agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="snowfall-agent",
    instructions="You are a weather assistant tracking snowfall. Use the provided functions to answer questions. When the user asks about snowfall, use the recent_snowfall function.",
    toolset=toolset
)

In [22]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Hello, send  information about Denver?",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Created thread, ID: thread_VoPTgFYCNlkW25gEBHkQRtMU
Created message, ID: msg_V9pVP7r8LVnJFKOb6oVOgwJr
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_p09vTUFHWzbUKyhpJU4y4pAn', 'object': 'thread.message', 'created_at': 1745167227, 'assistant_id': 'asst_H0XyQsstdKpbyDrP1AG82s2r', 'thread_id': 'thread_VoPTgFYCNlkW25gEBHkQRtMU', 'run_id': 'run_DMnQcaItidcVvUd0bikNxKeO', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Recent snowfall in Denver is 2 inches. If you need more information, feel free to ask!', 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_V9pVP7r8LVnJFKOb6oVOgwJr', 'object': 'thread.message', 'created_at': 1745167222, 'assistant_id': None, 'thread_id': 'thread_VoPTgFYCNlkW25gEBHkQRtMU', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'Hello, send  information about Denver?', 'annotations': []}}], 'attachments': [], 'metadata': {}}], 'first_id

In [23]:
messages.data[0].content[0].text.value

'Recent snowfall in Denver is 2 inches. If you need more information, feel free to ask!'